In [ ]:
import sklearn
import numpy as np
import pandas as pd
import os, time, re, json
from sklearn.model_selection import train_test_split

In [ ]:
input_articles_pmcids = "scripts/exp_input/REV.txt"

ground_truth_complete = "scripts/Local_model_finetuning/ground_truth/gt_dataset_info_extraction_from_snippet.xlsx"
ground_truth_no_dspage = "scripts/Local_model_finetuning/ground_truth/gt_dataset_info_no_dspage_extraction_from_snippet.xlsx"

In [ ]:
with open(input_articles_pmcids, 'r') as f:
    pmc_links = f.read().splitlines()

print("Total number of PMCIDs:", len(pmc_links))


train_pmc_links, test_pmc_links = train_test_split(pmc_links, test_size=0.2, random_state=42)

print(f"Training set: {len(train_pmc_links)}")
print(f"Test set: {len(test_pmc_links)}")

In [ ]:
train_test_df = pd.read_excel(ground_truth_no_dspage)

In [ ]:
# Split the DataFrame based on train and test PMC links
train_df = train_test_df[train_test_df['url'].isin(train_pmc_links)]
test_df = train_test_df[train_test_df['url'].isin(test_pmc_links)]

print(f"Original DataFrame: {len(train_test_df)} rows")
print(f"Train DataFrame: {len(train_df)} rows")
print(f"Test DataFrame: {len(test_df)} rows")
print(f"Total matched: {len(train_df) + len(test_df)} rows")

In [ ]:
# Examine the input and output text to understand the task
print("Sample input_text:")
print(train_df['input_text'].iloc[0][:500])
print("\n" + "="*80 + "\n")
print("Sample output_text:")
print(train_df['output_text'].iloc[0][:500])
print("\n" + "="*80 + "\n")
print(f"Average input length: {train_df['input_text'].str.len().mean():.0f} chars")
print(f"Average output length: {train_df['output_text'].str.len().mean():.0f} chars")

# Model Training Strategy

## Task: Dataset Information Extraction
Extract structured dataset identifiers and repository references from scientific text.

#### Approach: **Fine-tuned T5/Flan-T5** (Recommended)
- Designed for text-to-text generation
- Good at structured output
- Fast inference
- Model: `google/flan-t5-base` (250M params)

In [ ]:
# Prepare data in the format needed for training
from datasets import Dataset

def prepare_dataset(df):
    """Convert DataFrame to HuggingFace Dataset format, filtering out NaN values"""
    # Remove rows where input_text or output_text is NaN
    df_clean = df.dropna(subset=['input_text', 'output_text']).copy()
    
    # Convert to string to ensure all values are strings
    df_clean['input_text'] = df_clean['input_text'].astype(str)
    df_clean['output_text'] = df_clean['output_text'].astype(str)
    
    data = {
        'input': df_clean['input_text'].tolist(),
        'output': df_clean['output_text'].tolist()
    }
    
    print(f"  Filtered out {len(df) - len(df_clean)} rows with missing values")
    return Dataset.from_dict(data)

print("Preparing train dataset...")
train_dataset = prepare_dataset(train_df)

print("\nPreparing test dataset...")
test_dataset = prepare_dataset(test_df)

print(f"\nTrain dataset: {len(train_dataset)} examples")
print(f"Test dataset: {len(test_dataset)} examples")
print("\nSample:")
print(f"Input: {train_dataset[0]['input'][:200]}...")
print(f"Output: {train_dataset[0]['output']}")

# Fine-tune Flan-T5

This approach uses:
- **Model**: `google/flan-t5-base` - instruction-tuned T5 model
- **Training**: Parameter-efficient fine-tuning
- **Framework**: HuggingFace Transformers + Trainer API

In [ ]:
# Install required packages (run once)
!pip install -q transformers datasets accelerate evaluate rouge-score sentencepiece

In [ ]:
from transformers import (
    AutoTokenizer, 
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq
)
import evaluate
import numpy as np

# Load model and tokenizer
model_name = "google/flan-t5-base"  # 250M parameters
print(f"Loading model: {model_name}")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

print(f"Model loaded: {model.num_parameters():,} parameters")

In [ ]:
# Tokenize the datasets
def preprocess_function(examples):
    """Tokenize inputs and outputs"""
    # Add task prefix to help the model understand the task
    inputs = ["Extract dataset information: " + doc for doc in examples['input']]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding=False)
    
    # Tokenize targets
    labels = tokenizer(text_target=examples['output'], max_length=256, truncation=True, padding=False)
    model_inputs["labels"] = labels["input_ids"]
    
    return model_inputs

# Tokenize datasets
tokenized_train = train_dataset.map(preprocess_function, batched=True, remove_columns=train_dataset.column_names)
tokenized_test = test_dataset.map(preprocess_function, batched=True, remove_columns=test_dataset.column_names)

print(f"Tokenized train dataset: {len(tokenized_train)} examples")
print(f"Tokenized test dataset: {len(tokenized_test)} examples")

In [ ]:
# Setup evaluation metrics
rouge = evaluate.load("rouge")

def compute_metrics(eval_preds):
    """Compute ROUGE scores for evaluation"""
    preds, labels = eval_preds
    
    # Decode predictions
    if isinstance(preds, tuple):
        preds = preds[0]
    
    # Replace -100 in labels (padding token)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Compute ROUGE scores
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    
    # Extract F1 scores
    result = {k: round(v * 100, 2) for k, v in result.items()}
    
    # Compute exact match (for structured output)
    exact_match = sum([p.strip() == l.strip() for p, l in zip(decoded_preds, decoded_labels)]) / len(decoded_preds)
    result["exact_match"] = round(exact_match * 100, 2)
    
    return result

In [ ]:
# Setup training arguments
output_dir = "./results/flan-t5-dataset-extraction"

training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    
    # Training hyperparameters
    num_train_epochs=5,
    learning_rate=3e-4,
    per_device_train_batch_size=4,  # Reduced from 8 to 4 for memory
    per_device_eval_batch_size=4,   # Reduced from 8 to 4 for memory
    gradient_accumulation_steps=4,  # Increased to 4 to maintain effective batch size = 16
    
    # Evaluation and logging
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
    
    # Generation settings for evaluation
    predict_with_generate=True,
    generation_max_length=256,
    
    # Optimizer settings
    weight_decay=0.01,
    warmup_steps=100,
    
    # Save settings
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="exact_match",
    greater_is_better=True,
    
    # Performance
    fp16=False,  # Set to True if you have GPU with fp16 support
    
    # Reproducibility
    seed=42,
)

print(f"Training configuration:")
print(f"  Epochs: {training_args.num_train_epochs}")
print(f"  Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"  Learning rate: {training_args.learning_rate}")
print(f"  Output directory: {output_dir}")

In [ ]:
# Initialize data collator
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# Initialize trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("Trainer initialized successfully!")
print(f"Training samples: {len(tokenized_train)}")
print(f"Evaluation samples: {len(tokenized_test)}")

In [ ]:
# Train the model
print("Starting training...")
train_result = trainer.train()

# Print training summary
print("\nTraining completed!")
print(f"Training loss: {train_result.training_loss:.4f}")
print(f"Training time: {train_result.metrics['train_runtime']:.2f} seconds")

# Save the final model
trainer.save_model(f"{output_dir}/final_model")
tokenizer.save_pretrained(f"{output_dir}/final_model")
print(f"\nModel saved to {output_dir}/final_model")

In [ ]:
# Evaluate on test set
print("Evaluating on test set...")
eval_results = trainer.evaluate()

print("\nTest Set Results:")
for key, value in eval_results.items():
    print(f"  {key}: {value}")
    
# Test inference on a few examples
print("\n" + "="*80)
print("Sample Predictions:")
print("="*80)

for i in range(min(3, len(test_dataset))):
    input_text = test_dataset[i]['input']
    expected_output = test_dataset[i]['output']
    
    # Prepare input
    input_ids = tokenizer(
        "Extract dataset information: " + input_text,
        return_tensors="pt",
        max_length=512,
        truncation=True
    ).input_ids
    
    # Generate prediction
    outputs = model.generate(
        input_ids,
        max_length=256,
        num_beams=4,
        early_stopping=True
    )
    
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    print(f"\nExample {i+1}:")
    print(f"Input: {input_text[:200]}...")
    print(f"Expected: {expected_output}")
    print(f"Predicted: {prediction}")
    print(f"Match: {'✓' if prediction.strip() == expected_output.strip() else '✗'}")